# Mercedes Benz Greener Prediction 

Final.ipynb file should contain the following :

It should have two functions.

Function-1

Should include entire pipeline, from data preprocessing to making final predictions.

It should take in raw data as input.

It should return predictions for your input. Here the input can be a single point or a set of points.

def final_fun_1(X):

.....

..... # you will use the best model that you found out with your experiments
return predictions made on X ( Raw Data)

Function-2

Should include entire pipeline, from data preprocessing to making final predictions.

It should take in raw data as input along with its target values.

It should return the metric value that you are judging your models on.

def final_fun_2(X,Y):
.....

..... # you will use the best model that you found out with your experiments
return final_metric computed on X ( Raw Data) and Y (target variable)

# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import boxcox
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
from sklearn.preprocessing import PolynomialFeatures
from scipy.stats import boxcox
from scipy.special import inv_boxcox
import pickle
from sklearn.decomposition import PCA
import copy

# Function1

In [3]:
train_data=pd.read_csv('train.csv')
X=train_data.drop('y',axis=1)
y=train_data['y']

In [7]:
def final_func_1(X):
  """This function will preprocess all  data and 
  return prediction for a individual point
  X: individual point
  """
  #Dropping all the features which are having low variance and zero variance
  file=open('drop_features','rb')
  drop_column=pickle.load(file)
  file.close()
  X.drop(drop_column,axis=1,inplace=True)
  #Categorical encoding
  file=open('Encoder','rb')
  encoder=pickle.load(file)
  file.close()
  #Storing all the column names which are categorical
  cat_columns=X.select_dtypes(include='object').columns 
  X[cat_columns]=encoder.fit_transform(X[cat_columns])
  #Training Pca on all features except categorical features.
  file=open('PCA','rb')
  pca=pickle.load(file)
  file.close()
  X_pca_feat=pca.transform(X.drop(labels=cat_columns,axis=1)) 
  #Creating DataFrame for Pca components with column names, so that we can merge this dataset with original dataset later.
  X_pca_df=pd.DataFrame(X_pca_feat,columns=['pca1','pca2','pca3','pca4','pca5','pca6','pca7','pca8','pca9','pca10'])
  #Polynomial Features
  file=open('Polynomial_Features','rb')
  trans=pickle.load(file)
  file.close()
  X_poly=trans.transform(X[['X314','X315','X118']])
  poly_columns=trans.get_feature_names(X[['X314','X315','X118']].columns)
  X_polynomial_df=pd.DataFrame(X_poly,columns=poly_columns)
  X_polynomial_df.drop(labels=['X314','X315','X118'],axis=1,inplace=True)
  X=pd.concat([X,X_pca_df,X_polynomial_df],axis=1)
  file=open('best_model','rb')
  best_model=pickle.load(file)
  file.close()
  y_pred=best_model.predict(X)
  y_pred=inv_boxcox(y_pred,-0.6935216932798669)
  addition_func=lambda x:x+1
  y_pred=addition_func(y_pred)
  return y_pred



In [8]:
x=X.loc[[0]]

In [9]:
x

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,...,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,k,v,at,a,d,u,j,o,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [10]:
predictions=final_func_1(x)

[09:41:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [11]:
print('Prediction for the a query point is ',predictions)

Prediction for the a query point is  [107.586174]


# Function2

In [43]:
def final_func_2(X,y):
  """This function will preprocess all the data and 
  return Metric score(R-squared value) for a set of points"""
   #Dropping all the features which are having low variance and zero variance
  file=open('drop_features','rb')
  drop_column=pickle.load(file)
  file.close()
  X.drop(drop_column,axis=1,inplace=True)
  #Categorical encoding
  file=open('Encoder','rb')
  encoder=pickle.load(file)
  file.close()
  #Storing all the column names which are categorical
  cat_columns=X.select_dtypes(include='object').columns 
  X[cat_columns]=encoder.fit_transform(X[cat_columns])
  #Training Pca on all features except categorical features.
  file=open('PCA','rb')
  pca=pickle.load(file)
  file.close()
  X_pca_feat=pca.transform(X.drop(labels=cat_columns,axis=1)) 
  #Creating DataFrame for Pca components with column names, so that we can merge this dataset with original dataset later.
  X_pca_df=pd.DataFrame(X_pca_feat,columns=['pca1','pca2','pca3','pca4','pca5','pca6','pca7','pca8','pca9','pca10'])
  #Polynomial Features
  file=open('Polynomial_Features','rb')
  trans=pickle.load(file)
  file.close()
  X_poly=trans.transform(X[['X314','X315','X118']])
  poly_columns=trans.get_feature_names(X[['X314','X315','X118']].columns)
  X_polynomial_df=pd.DataFrame(X_poly,columns=poly_columns)
  X_polynomial_df.drop(labels=['X314','X315','X118'],axis=1,inplace=True)
  X=pd.concat([X,X_pca_df,X_polynomial_df],axis=1)
  file=open('best_model','rb')
  best_model=pickle.load(file)
  file.close()
  y_pred=best_model.predict(X)
  y_boxcox=boxcox(y.values.flatten(),lmbda=-0.6935216932798669)
  R_squared=r2_score(y_boxcox,y_pred)
  return R_squared


In [44]:
#Splitting the data to check R-squared value
X_train,X_cv,y_train,y_cv=train_test_split(X,y,test_size=0.2,random_state=0)
X_cv.reset_index(drop=True,inplace=True)
y_cv.reset_index(drop=True,inplace=True)


In [45]:
R_squared=final_func_2(X_cv,y_cv)

[10:04:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [46]:
print("R_squared value obtained is ",R_squared)

R_squared value obtained is  0.6562838348652362
